In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import glob
import plotly.io as pio

# Open Plotly charts in a separate browser window/tab
pio.renderers.default = "browser"

base_folder = Path(r"C:\Users\phuynh\Projects\robotray\from_hugo")
folders = [p for p in base_folder.iterdir() if p.is_dir()]

print("Folders:", [p.name for p in folders])

# Create subplots: 2 rows x 3 cols
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        "Mining High Voltage Spectra",
        "Mining Low Voltage Spectra",
        "",
        "Soil High Voltage Spectra",
        "Soil Mid Voltage Spectra",
        "Soil Low Voltage Spectra"
    ),
    x_title="Energy (keV)",
    y_title="Intensity (CPS)",
    specs=[[{}, {}, {}], [{}, {}, {}]]
)

# Helper function to determine app type (Mining or Soil)
def get_app_type(filename):
    if "Mining" in filename:
        return "Mining"
    elif "Soil" in filename:
        return "Soil"
    return None

# Helper function to get the correct intensity column name
def get_intensity_column(df):
    if 'Intensity (CPS)' in df.columns:
        return 'Intensity (CPS)'
    if 'Intensity (cps)' in df.columns:
        return 'Intensity (cps)'
    return None

# Helper function to check if file has spectral data
def has_spectral_data(df):
    return 'Energy (keV)' in df.columns and get_intensity_column(df) is not None

# Helper for median + IQR band
def add_median_band(frames, row, col, label, rgb):
    if not frames:
        return
    merged = frames[0]
    for frame in frames[1:]:
        merged = merged.merge(frame, on="Energy (keV)", how="inner")

    energy = merged["Energy (keV)"]
    values = merged.drop(columns=["Energy (keV)"])
    median = values.median(axis=1)
    q25 = values.quantile(0.25, axis=1)
    q75 = values.quantile(0.75, axis=1)

    fig.add_trace(
        go.Scatter(
            x=energy,
            y=q75,
            mode="lines",
            line=dict(width=0),
            showlegend=False,
            hoverinfo="skip"
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=energy,
            y=q25,
            mode="lines",
            fill="tonexty",
            fillcolor=f"rgba({rgb[0]},{rgb[1]},{rgb[2]},0.25)",
            line=dict(width=0),
            name=f"{label} 25–75%",
            legendgroup=label
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=energy,
            y=median,
            mode="lines",
            line=dict(width=3, color=f"rgb({rgb[0]},{rgb[1]},{rgb[2]})"),
            name=f"{label} median",
            legendgroup=label
        ),
        row=row, col=col
    )

# Colors per folder
palette = [
    (31, 119, 180),  # blue
    (255, 127, 14)   # orange
]

for idx, folder in enumerate(folders):
    csv_files = sorted(glob.glob(str(folder / "*.csv")))
    if not csv_files:
        continue

    color = palette[idx % len(palette)]
    label_prefix = folder.name

    mining_hv = []
    mining_lv = []
    soil_hv = []
    soil_mid = []
    soil_lv = []

    for csv_file in csv_files:
        filename = Path(csv_file).name

        # Skip files containing "chemistry" or "photo"
        if "chemistry" in filename.lower() or "photo" in filename.lower():
            continue

        df = pd.read_csv(csv_file)

        # Skip files without spectral data
        if not has_spectral_data(df):
            continue

        intensity_col = get_intensity_column(df)
        app_type = get_app_type(filename)

        if app_type == "Mining":
            if "HighVoltage" in filename:
                mining_hv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: filename}))
            elif "LowVoltage" in filename:
                mining_lv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: filename}))
        elif app_type == "Soil":
            if "HighVoltage" in filename:
                soil_hv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: filename}))
            elif "MidVoltage" in filename:
                soil_mid.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: filename}))
            elif "LowVoltage" in filename:
                soil_lv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: filename}))

    add_median_band(mining_hv, 1, 1, f"{label_prefix} Mining HV", color)
    add_median_band(mining_lv, 1, 2, f"{label_prefix} Mining LV", color)
    add_median_band(soil_hv, 2, 1, f"{label_prefix} Soil HV", color)
    add_median_band(soil_mid, 2, 2, f"{label_prefix} Soil Mid", color)
    add_median_band(soil_lv, 2, 3, f"{label_prefix} Soil LV", color)

# Update axis labels
for row in [1, 2]:
    for col in [1, 2, 3]:
        fig.update_xaxes(title_text="Energy (keV)", row=row, col=col)
        fig.update_yaxes(title_text="Intensity (CPS)", row=row, col=col)

fig.update_layout(height=900, width=1600)
fig.show()

Folders: ['nephe_150', 'none_149']
